<a href="https://colab.research.google.com/github/gazalssr/HerdNet/blob/main/Correct_One_Phase_Finetunning%2BBinary_head.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DEMO - Training and testing HerdNet on nadir aerial images
test

## Installations

In [ ]:
# Check GPU
!nvidia-smi

In [ ]:
# Install the dependencies
!pip install h5py
!pip install typing-extensions
!pip install wheel
!pip install albumentations>=1.0.3
!pip install fiftyone>=0.14.3
!pip install hydra-core>=1.1.0
!pip install opencv-python>=4.5.1.48
!pip install pandas>=1.2.3
!pip install pillow>=8.2.0
!pip install scikit-image>=0.18.1
!pip install scikit-learn>=1.0.2
!pip install scipy>=1.6.2
!pip install wandb>=0.10.33
!pip install numpy>=1.20.0


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Download and install the code
import sys

!git clone https://github.com/gazalssr/HerdNet.git
#!cd '/content/HerdNet' && python setup.py install
!pip install -e /content/HerdNet
sys.path.append('/content/HerdNet')

In [ ]:
!wandb login
import wandb

## Create datasets

In [ ]:
# Set the seed
from animaloc.utils.seed import set_seed

set_seed(9292)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import PIL.Image as Image

from animaloc.data.patches import ImageToPatches, AnnotatedImageToPatches

img_path = '//content/drive/MyDrive/CAH_small_samples/Train_CAH_Sample/CAH_7_90_2017_BR.jpg'

sample_img = Image.open(img_path)

patcher = ImageToPatches(sample_img, (512,512), overlap = 100)
patches = patcher.make_patches()

print(f'Number of patches: {len(patches)}')

plt.figure(figsize=(15,10))
_ = patcher.show()

In [ ]:
# Create training  patches using the patcher tool
from animaloc.utils.useful_funcs import mkdir
import pandas as pd
from PIL import Image
df = pd.read_csv('/content/drive/MyDrive/Small_CAH_whole/train/train.csv')
df = df.to_csv('/content/drive/MyDrive/Small_CAH_whole/train/train.csv', index=False)
# df = pd.read_csv('/content/drive/MyDrive/CAH_small_samples/Train_CAH_Sample/Train_CAH_Sample.csv')
# df = df.to_csv('/content/drive/MyDrive/CAH_small_samples/Train_CAH_Sample/Train_CAH_Sample.csv', index=False)
mkdir('/content/drive/MyDrive/Train_patches_stratified')
#!python /content/HerdNet/tools/patcher.py /content/drive/MyDrive/CAH_small_samples/Train_CAH_Sample/ 512 512 256 /content/drive/MyDrive/Train_patches -csv /content/drive/MyDrive/CAH_small_samples/Train_CAH_Sample/Train_CAH_Sample.csv -min 0.0 -all False
!python /content/HerdNet/tools/patcher.py  /content/drive/MyDrive/Small_CAH_whole/train 512 512 256  /content/drive/MyDrive/Train_patches_stratified -csv /content/drive/MyDrive/Small_CAH_whole/train/train.csv -min 0.0 -all False

In [ ]:
# Create validation patches
from animaloc.utils.useful_funcs import mkdir
import pandas as pd
# mkdir('/content/drive/MyDrive/val_patches')
mkdir('/content/drive/MyDrive/val_patches_stratified')
# !python /content/HerdNet/tools/patcher.py /content/drive/MyDrive/CAH_training_session_1/ValCAH 512 512 256 /content/drive/MyDrive/val_patches -csv /content/drive/MyDrive/CAH_training_session_1/ValCAH/ValCAH_2017_1.csv -min 0.0 -all True
!python /content/HerdNet/tools/patcher.py /content/drive/MyDrive/Small_CAH_whole/val/ 512 512 256 /content/drive/MyDrive/val_patches_stratified -csv /content/drive/MyDrive/Small_CAH_whole/val/val.csv -min 0.0 -all False

In [ ]:
%pdb

In [ ]:
# Create test patches
from animaloc.utils.useful_funcs import mkdir
import pandas as pd

# mkdir('/content/drive/MyDrive/test_patches')
mkdir('/content/drive/MyDrive/test_patches_stratified')
!python /content/HerdNet/tools/patcher.py /content/drive/MyDrive/Small_CAH_whole/test/ 512 512 256 /content/drive/MyDrive/test_patches_stratified -csv /content/drive/MyDrive/Small_CAH_whole/test/test.csv -min 0.0 -all False

## Caribou dataset donwload

In [ ]:
#make a zip file from the patch folders
#Zip file path (destination)       source file path
%cd /content/drive/MyDrive/
# Zip the directory
!zip -jr 'Stratified_patches_zip/Train_patches_stratified.zip' 'Train_patches_stratified'
!zip -jr 'Stratified_patches_zip/val_patches_stratified.zip' 'val_patches_stratified'
!zip -jr 'Stratified_patches_zip/test_patches_stratified.zip' 'test_patches_stratified'

# import shutil
# shutil.make_archive('/content/drive/MyDrive/Train_patches_S', 'zip', '/content/drive/MyDrive/Train_patches')
# shutil.make_archive('/content/drive/MyDrive/val_patches_S', 'zip', '/content/drive/MyDrive/val_patches')
# shutil.make_archive('/content/drive/MyDrive/test_patches_S', 'zip', '/content/drive/MyDrive/test_patches')

In [ ]:
#### Downloading and unziping the files
#zip file download (destination link)
# %cd /content/drive/MyDrive/
!pip install --upgrade --no-cache-dir gdown
# Download the Train zip file

!gdown https://drive.google.com/uc?id=1mI6Ve5v3sAj9h502g75GD1lZSYy4-FR4 -O /content/Train_patches_stratified.zip
# Unzip the file to the specified directory
!unzip -oq /content/Train_patches_stratified.zip -d /content/Train_patches_stratified

# Download the val zip file
!gdown https://drive.google.com/uc?id=1-1lGSZVk-ts0TMo0n-sbwlBGKHhgh9O9 -O /content/val_patches_stratified.zip
# Unzip the file to the specified directory
!unzip -oq /content/val_patches_stratified.zip -d /content/val_patches_stratified

# Download the test zip file
!gdown https://drive.google.com/uc?id=1-1r9sQlC-NxgcSvKKl0WPEmOpkzRV4KB -O /content/test_patches_stratified.zip

# Unzip the file to the specified directory
!unzip -oq /content/test_patches_stratified.zip -d /content/test_patches_stratified



In [ ]:
# Download the val zip file
!gdown https://drive.google.com/uc?id=1-1lGSZVk-ts0TMo0n-sbwlBGKHhgh9O9 -O /content/val_patches_stratified.zip
# Unzip the file to the specified directory
!unzip -oq /content/val_patches_stratified.zip -d /content/val_patches_stratified

In [ ]:

!unzip -l /content/drive/MyDrive/Stratified_patches_zip/Train_patches_stratified.zip


In [ ]:
%matplotlib inline
# Showing some samples of patches and the annotations
import matplotlib.pyplot as plt
from animaloc.datasets import CSVDataset
from animaloc.data.batch_utils import show_batch, collate_fn
from torch.utils.data import DataLoader
import torch
import albumentations as A
batch_size = 8
csv_path = '/content/Train_patches_stratified/gt.csv'
image_path = '/content/Train_patches_stratified'
dataset = CSVDataset(csv_path, image_path, [A.Normalize()])
dataloader = DataLoader(dataset, batch_size=batch_size, collate_fn=collate_fn, shuffle=True)

sample_batch = next(iter(dataloader))
for i in range(len(sample_batch[1])):
  points = sample_batch[1][i]['points'].numpy()
  bbox= []
  for pt in points:
      bbox.append([pt[0]-2,pt[1]-2,pt[0]+2,pt[1]+2])
  print(len(sample_batch[1][i]['labels']))
  sample_batch[1][i]['annotations']=torch.tensor(bbox)
plt.figure(figsize=(16,2))
show_batch(sample_batch)

In [ ]:
# Training, validation and test datasets
import albumentations as A

from animaloc.datasets import FolderDataset
from animaloc.data.transforms import MultiTransformsWrapper, DownSample, PointsToMask, FIDT

patch_size = 512
num_classes = 2
down_ratio: int = 2

train_dataset = CSVDataset(

    csv_file = '/content/Train_patches_stratified/gt.csv',
    root_dir = '/content/Train_patches_stratified',
    # Data Augmentation
    albu_transforms = [
        A.VerticalFlip(p=0.5),
        A.HorizontalFlip(p=0.5),
        A.Normalize(p=1.0),
        A.RandomRotate90(p=0.5),
        A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.2),
        A.Blur(blur_limit=15, p=0.2),
        ],
    end_transforms = [MultiTransformsWrapper([
        FIDT(num_classes=num_classes, down_ratio=down_ratio),
        PointsToMask(radius=2, num_classes=num_classes, squeeze=True, down_ratio=int(patch_size//16))
        ])]
    )

val_dataset = CSVDataset(

    csv_file = '/content/val_patches_stratified/gt.csv',
    root_dir = '/content/val_patches_stratified',
    albu_transforms = [A.Normalize(p=1.0)],
    end_transforms = [DownSample(down_ratio=down_ratio, anno_type='point')]
    )

test_dataset = CSVDataset(

    csv_file = '/content/test_patches_stratified/gt.csv',
    root_dir = '/content/test_patches_stratified',
    albu_transforms = [A.Normalize(p=1.0)],
    end_transforms = [DownSample(down_ratio=down_ratio, anno_type='point')]
    )

In [ ]:
# Dataloaders
from torch.utils.data import DataLoader
train_dataloader = DataLoader(dataset = train_dataset, batch_size = 4, shuffle = True)
val_dataloader = DataLoader(dataset = val_dataset, batch_size = 1, shuffle = False)
test_dataloader = DataLoader(dataset = test_dataset, batch_size = 1, shuffle = False)

## Define HerdNet for training

In [ ]:
# Path to your .pth file
import torch
####################### Transfered weights ###########################
pth_path = '/content/drive/MyDrive/output/Binary.pth'
pretrained_dict = torch.load(pth_path)

In [ ]:
from animaloc.models import HerdNet
from torch import Tensor
from animaloc.models import LossWrapper
from animaloc.train.losses import FocalLoss
from torch.nn import CrossEntropyLoss

herdnet = HerdNet(num_classes=num_classes, down_ratio=down_ratio).cuda()

losses = [
    {'loss': FocalLoss(reduction='mean'), 'idx': 0, 'idy': 0, 'lambda': 1.0, 'name': 'focal_loss'},
    {'loss': CrossEntropyLoss(reduction='mean'), 'idx': 1, 'idy': 1, 'lambda': 1.0, 'name': 'ce_loss'}
    ]

herdnet = LossWrapper(herdnet, losses=losses)
herdnet.load_state_dict(pretrained_dict, strict=False) # /initialisation from a pre-task

## Create the Trainer

In [ ]:
#############Get model layers ###########################
def get_parameter_names(model): # getting the model layers
  param_dict= dict()
  for l, (name,param) in enumerate(model.named_parameters()):
    #print(l,":\t",name,type(param),param.requires_grad)
    param_dict[name]= l
  return param_dict
result = get_parameter_names(herdnet)
print(result)

In [ ]:
#Freeze the layers
def freeze_parts(model, get_parameter_names, layers_to_freeze, lr, unfreeze=False):
    params_to_update = []

    for l, (name, param) in enumerate(model.named_parameters()):
        res = any(ele in name for ele in layers_to_freeze)
        param.requires_grad = unfreeze if res else not unfreeze

        if param.requires_grad == True:
            params_to_update.append({
                "params": param,
                "lr": lr,
            })

        # Print parameters to update
        if param.requires_grad:
            print(f"Trainable parameter: {name}")
        else:
            print(f"Frozen parameter: {name}")

    return params_to_update


In [ ]:
#Freeze half of a specified layer
def freeze_parts(model, get_parameter_names, layers_to_freeze, freeze_layer_half=None, lr=0.0001, unfreeze=False):
    params_to_update = []

    for l, (name, param) in enumerate(model.named_parameters()):
        res = any(ele in name for ele in layers_to_freeze)
        param.requires_grad = unfreeze if res else not unfreeze

        # Check if the current layer is the specified layer to freeze half of its parameters
        if freeze_layer_half is not None and freeze_layer_half in name:
            total_params = param.numel()
            half_params = total_params // 2
            param.requires_grad = unfreeze if l < half_params else not unfreeze

        if param.requires_grad:
            params_to_update.append({
                "params": param,
                "lr": lr,
            })

        # Print parameters to update
        if param.requires_grad:
            print(f"Trainable parameter: {name}")
        else:
            print(f"Frozen parameter: {name}")

    return params_to_update


In [ ]:
#Freeze half of the last layer
def freeze_parts(model, get_parameter_names, layers_to_freeze, lr, unfreeze=False, freeze_last_half=False):
    # Initialize an empty list to store parameters to update
    params_to_update = []

    # Iterate through all named parameters in the model
    for name, param in model.named_parameters():
        # Check if the parameter should be frozen based on the layers_to_freeze list
        should_freeze = any(ele in name for ele in layers_to_freeze)

        # If freeze_last_half is True, freeze half of the parameters in the last layer
        if freeze_last_half and 'your_last_layer_keyword' in name:
            # Calculate the number of channels in the last layer
            num_channels = param.size(0)

            # Calculate the number of channels to freeze (half of the total channels)
            num_channels_to_freeze = num_channels // 2

            # Check if the current channel should be frozen
            should_freeze = should_freeze or (param.size(0) >= num_channels - num_channels_to_freeze)

        # Set requires_grad attribute based on whether to freeze or unfreeze
        param.requires_grad = unfreeze if should_freeze else not unfreeze

        # Append parameters to the update list if requires_grad is True
        if param.requires_grad:
            params_to_update.append({
                "params": param,
                "lr": lr,
            })

        # Print information about the parameter (whether it's trainable or frozen)
        if param.requires_grad:
            print(f"Trainable parameter: {name}")
        else:
            print(f"Frozen parameter: {name}")

    return params_to_update


In [ ]:
#freezing half of one lyer+ other layers
params_to_update = freeze_parts(herdnet.model, get_parameter_names, layers_to_freeze=['base_layer','level0','level1','level2','level3'], freeze_layer_half='level_4', lr=0.0001, unfreeze=False)


In [ ]:
#Freezing a whole layer
lr=1e-4
# layers_to_freeze= [] # nothing frozen
# layers_to_freeze= ['base_layer','level0','level1','level2','level3','level4','level5'] # we are feezing all the levels below level5
layers_to_freeze= ['base_layer','level0','level1','level2','level3','level4']

params_to_update= freeze_parts(herdnet.model,get_parameter_names,layers_to_freeze,lr,unfreeze=False)

# Create the Trainer

In [ ]:
from torch.optim import Adam
from animaloc.train import Trainer
from torch.optim.lr_scheduler import StepLR
from animaloc.eval import PointsMetrics, HerdNetStitcher, HerdNetEvaluator
from animaloc.utils.useful_funcs import mkdir

work_dir = '/content/drive/MyDrive/output'
mkdir(work_dir)

lr = 1e-4
weight_decay = 1e-3
epochs = 100
# parameters= herdnet.parameters()
optimizer = Adam(params_to_update, lr=lr, weight_decay=weight_decay)
# scheduler = StepLR(optimizer, step_size=5, gamma=0.5)
# scheduler.step()
metrics = PointsMetrics(radius=20, num_classes=num_classes)

stitcher = HerdNetStitcher(
    model=herdnet,
    size=(patch_size,patch_size),
    overlap=0,
    down_ratio=down_ratio,
    reduction='mean'
    )

evaluator = HerdNetEvaluator(
    model=herdnet,
    dataloader=val_dataloader,
    metrics=metrics,
    stitcher=None,
    work_dir=work_dir,
    header='validation'
    )

trainer = Trainer(
    model=herdnet,
    train_dataloader=train_dataloader,
    optimizer=optimizer,
    num_epochs=epochs,
    evaluator=evaluator,             # metric evaluation
    # val_dataloader= val_dataloader, # loss evaluation
    work_dir=work_dir
    )

# Start Training

In [ ]:
import wandb
if wandb.run is not None:
  wandb.finish()
wandb.init(project="herdnet-finetuning")

In [ ]:
%pdb

In [ ]:
trainer.start(warmup_iters=100, checkpoints='best', select='max', validate_on='f1_score', wandb_flag =True)
# validate_on='f1_score'

In [ ]:
# df= evaluator.results # getting the validation results in pandas format
# df.head()
evaluator.metrics.confusion_matrix

In [ ]:
#save and load finetunned parameters
herdnet = HerdNet()
torch.save(herdnet.state_dict(), 'fine_tuned_model.pth')
herdnet.load_state_dict(torch.load('fine_tuned_model.pth'))
pth_path = '/content/drive/MyDrive/output/fine_tuned_model.pth'
torch.save(herdnet.state_dict(), pth_path)

## Test the model

In [ ]:
# Path to your .pth file
# pth_path = '/content/fine_tuned_model.pth'
finetuned_dict = torch.load(pth_path)

In [ ]:
# Create output folder
from animaloc.utils.useful_funcs import mkdir
test_dir = '/content/drive/MyDrive/test_output_one_phase_Finetune'
mkdir(test_dir)

In [ ]:
# Create an Evaluator
test_evaluator = HerdNetEvaluator(
    model=herdnet,
    dataloader=test_dataloader,
    metrics=metrics,
    stitcher=stitcher,
    work_dir=test_dir,
    header='test'
    )

In [ ]:
# Start testing
test_f1_score = test_evaluator.evaluate(returns='f1_score')

In [ ]:
# Print global F1 score (%)
print(f"F1 score = {test_f1_score * 100:0.0f}%")

In [ ]:
# Get the detections (Metrics)
detections = test_evaluator.results
detections.to_csv('/content//drive/MyDrive/test_output_pretrain/metrics.csv')

In [ ]:
# precision-recall curves
%matplotlib inline
from animaloc.vizual import PlotPrecisionRecall
pr_curve = PlotPrecisionRecall(legend=True)
metrics = test_evaluator._stored_metrics
for c in range(1, metrics.num_classes):
    rec, pre = metrics.rec_pre_lists(c)
    pr_curve.feed(rec, pre, str(c))

pr_curve.plot()

In [ ]:
# Train config files
%%writefile /content/HerdNet/configs/train/herdnet.yaml
wandb_project: 'hernet-finetunning'
wandb_entity: 'ghazaleh-serati'
wandb_run: 'myrun'
seed: 1
device_name: 'cuda'

model:
  name: 'HerdNet'
  from_torchvision: False
  load_from: null
  resume_from: null
  kwargs:
    num_layers: 34
    pretrained: True
    down_ratio: 2
    head_conv: 64
  freeze: null

losses:
  FocalLoss:
    print_name: 'focal_loss'
    from_torch: False
    output_idx: 0
    target_idx: 0
    lambda_const: 1.0
    kwargs:
      reduction: 'mean'
      normalize: False
  CrossEntropyLoss:
    print_name: 'ce_loss'
    from_torch: True
    output_idx: 1
    target_idx: 1
    lambda_const: 1.0
    kwargs:
      reduction: 'mean'
      weight: [0.1,5.,15.,1.]

datasets:
  img_size: [512,512]
  anno_type: 'point'
  num_classes: 1
  collate_fn: null

  class_def:
    1: 'caribou'


  train:
    name: 'CSVDataset'
    csv_file: '/content/Train_patches/gt.csv'
    root_dir: '/content/Train_patches'

    sampler: null

    albu_transforms:
      HorizontalFlip:
        p: 0.5
      MotionBlur:
        p: 0.5
      Normalize:
        p: 1.0

    end_transforms:
      MultiTransformsWrapper:
        FIDT:
          num_classes: ${train.datasets.num_classes}
          down_ratio: ${train.model.kwargs.down_ratio}
        PointsToMask:
          radius: 2
          num_classes: ${train.datasets.num_classes}
          squeeze: True
          down_ratio: 32

  validate:
    name: 'CSVDataset'
    csv_file: '/content/drive/MyDrive/val_patches/gt.csv'
    root_dir: '/content/drive/MyDrive/val_patches'

    albu_transforms:
      Normalize:
        p: 1.0

    end_transforms:
      DownSample:
        down_ratio: ${train.model.kwargs.down_ratio}
        anno_type: ${train.datasets.anno_type}

training_settings:
  trainer: 'Trainer'
  valid_freq: 1
  print_freq: 100
  batch_size: 4
  optimizer: 'adam'
  lr: 1e-4
  weight_decay: 0.0005
  auto_lr:
    mode: 'max'
    patience: 10
    threshold: 1e-4
    threshold_mode: 'rel'
    cooldown: 10
    min_lr: 1e-6
    verbose: True
  warmup_iters: 100
  vizual_fn: null
  epochs: 1
  evaluator:
    name: 'HerdNetEvaluator'
    threshold: 5
    select_mode: 'max'
    validate_on: 'f1_score'
    kwargs:
      print_freq: 10
      lmds_kwargs:
        kernel_size: [3,3]
        adapt_ts: 0.3
  stitcher:
    name: 'HerdNetStitcher'
    kwargs:
      overlap: 0
      down_ratio: ${train.model.kwargs.down_ratio}
      up: False
      reduction: 'mean'

In [ ]:
# TEST config files
%%writefile /content/HerdNet/configs/test/herdnet.yaml
wandb_project: 'HerdNet'
wandb_entity: 'ghazaleh-serati'
wandb_run: 'myrun'
device_name: 'cuda'

model:
  name: 'HerdNet'
  from_torchvision: False
  pth_file: '/content/drive/MyDrive/output/best_model.pth'
  kwargs:
    num_layers: 34
    pretrained: False
    down_ratio: 2
    head_conv: 64

dataset:
  img_size: [512,512]
  anno_type: 'point'
  num_classes: 1
  collate_fn: null

  class_def:
    1: 'caribou'


  name: 'CSVDataset'
  csv_file: '/content/test_patches/gt.csv'
  root_dir: '/content/test_patches'

  mean: [0.485, 0.456, 0.406]
  std: [0.229, 0.224, 0.225]

evaluator:
  name: 'HerdNetEvaluator'
  threshold: 5
  kwargs:
    lmds_kwargs:
      kernel_size: [3,3]
      adapt_ts: 0.3

stitcher:
  name: 'HerdNetStitcher'
  kwargs:
    overlap: 160
    down_ratio: ${test.model.kwargs.down_ratio}
    up: False
    reduction: 'mean'

In [ ]:
# Launching training and testing sessions with config files

!python /content/HerdNet/tools/train.py train=HerdNet

In [ ]:
!python /content/HerdNet/tools/train.py -m train=HerdNet train.training_settings.batch_size=1,2,4 train.training_settings.lr=1e-3,1e-4

In [ ]:
!python /content/HerdNet/tools/test.py test=HerdNet